## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Imports

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os

%matplotlib inline
#%matplotlib qt

## Global variables

In [2]:
path_camera_calibration = './camera_cal/'
path_test_images = './test_images/'
path_ouput_images = './output_images/'

pickle_file = 'wide_dist_pickle.p'
input_video = 'project_video.mp4'

# Arrays to store object points and image points from all the images.
obj_points = [] # 3d points in real world space
img_points = [] # 2d points in image plane.

# Make a list of calibration images
images_list = glob.glob(path_camera_calibration + 'calibration*.jpg')
chessboard_images = []
undistorted_images = []

# dictionary for creating binary file for calibration data
dist_pickle = {}

# check for camera calibration directory
if not os.path.exists(path_camera_calibration):
    os.makedirs(path_camera_calibration)
    
# check for test images directory
if not os.path.exists(path_test_images):
    os.makedirs(path_test_images)

# check for output images directory
if not os.path.exists(path_ouput_images):
    os.makedirs(path_ouput_images)

## Helper functions

In [3]:
def get_output_file_path(filename):
    '''creates output path for images and returns it'''
    return path_ouput_images + filename + '.jpg'

def save_output_file(img, filename):
    '''saves the image as file'''
    mpimg.imsave(get_output_file_path(filename), img)

## Pipeline functions

In [4]:
def calibration_undistort(img, obj_points, img_points):
    '''performs the camera calibration, image distortion correction and 
    returns the undistorted image'''
    
    # defined image size
    img_size = (img.shape[1], img.shape[0])
    
    # calibrate camera and return parameters 
    retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_size, None, None)
    
    # undistrot image based on calibration parameters
    undist = cv2.undistort(img, cameraMatrix, distCoeffs, None, cameraMatrix)
    
    return undist

## Camera Calibration

In [5]:
dimensions = (9,6)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_p = np.zeros((6*9, 3), np.float32)
obj_p[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

# Step through the list and search for chessboard corners
for fname in images_list:
    # Read each image
    img = mpimg.imread(fname)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, dimensions, None)

    # If corners are found, add object points, image points and image to images array
    if ret == True:
        obj_points.append(obj_p)
        img_points.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, dimensions, corners, ret)
        chessboard_images.append(img)

In [6]:
# save output image
save_output_file(chessboard_images[0], 'camera_calibration')

In [7]:
# save points to pickle file for later use
dist_pickle['obj_points'] = obj_points
dist_pickle['img_points'] = img_points
pickle.dump(dist_pickle, open(path_camera_calibration + pickle_file, 'wb'))

## Correction for Distortion

In [8]:
# Read in the saved points and points from pickle file for usage
dist_pickle = pickle.load(open(path_camera_calibration + pickle_file, 'rb'))
obj_points = dist_pickle['obj_points']
img_points = dist_pickle['img_points']

### Test calibration and distortion correction

In [9]:
test_images_list = glob.glob(path_test_images + 'test*.jpg')
undistorted_images = []

# read in images and append to images
for fname in test_images_list:
    # Read each image
    img = mpimg.imread(fname)
    
    # Undistort the image
    dst = calibration_undistort(img, obj_points, img_points)
    
    # append to list
    undistorted_images.append(dst)

In [10]:
# save output image
save_output_file(undistorted_images[0], 'raw_undistorted')

## Read in video and apply pipeline on it